import data and filesettings notebook

In [1]:
%run fMRI_prestats0_general.ipynb

FED047 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED037 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED006 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED022 
 More than 2 GRE_FIELD sequences (4) .... select only first two for TE extraction 

FED067 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED008 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 

FED023 
 More than 2 GRE_FIELD sequences (4) .... select only first two for TE extraction 

FED029 
 Not enough GRE_FIELD sequences to read out (0).... exclude/investigate 



 The following subjects where excluded from distortion correction due to false file numbers:  
 GRE:  ['FED006', 'FED008', 'FED029', 'FED037', 'FED047', 'FED067'] EPI:  []
FED022 
 More than 2 GRE_FIELD sequences (4) .... selecting first two for TE extraction 

FED066's json file does not specify DwellTime 
 noting issue

In [24]:
# control results
key= random.sample(data.keys(), 1)
print(data[key[0]])
print(data["FED012"])

{'files': ['/fMRI/FED067/FMRI_0007_20160311133810_7.json', '/fMRI/FED067/FMRI_0007_20160311133810_7.nii', '/fMRI/FED067/T1_MPRAGE_0005_20160311133810.json', '/fMRI/FED067/T1_MPRAGE_0005_20160311133810.nii'], 'parameters': {}}
{'files': ['/fMRI/FED012/FMRI_FS_0010_20141124150148_10.json', '/fMRI/FED012/FMRI_FS_0010_20141124150148_10.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e1.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e1.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e2.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0006_20141124150148_e2.nii', '/fMRI/FED012/GRE_FIELD_MAPPING_0007_20141124150148_e2_ph.json', '/fMRI/FED012/GRE_FIELD_MAPPING_0007_20141124150148_e2_ph.nii', '/fMRI/FED012/GRE_e1mag_bet_B.nii', '/fMRI/FED012/GRE_e1mag_bet_B_mask.nii', '/fMRI/FED012/GRE_fieldmap.nii', '/fMRI/FED012/T1_MPRAGE_0005_20141124150148.json', '/fMRI/FED012/T1_MPRAGE_0005_20141124150148.nii'], 'parameters': {'GRE_ET': [0.00492, 0.00738], 'GRE_PED': ['i'], 'EPI_ET': 

# Processing

Motion correct / realign

Slicetime correction (?)

I will omit this step as advised by Poldrack, Nichols and Mumford "Handbook of fMRI data analysis"(2011)\
TR= 1800ms  > 2seconds, interleaved acquisition\
event related analysis is relatively robust to slice-timing problems here.

Spatial normalisation

Spatial Smoothing

# QC